# LSTM Autoencoder for ICS Anomaly Detection\n\n**Industrial Control System (ICS) Time-Series Anomaly Detection Pipeline**\n\nThis notebook demonstrates a complete ML/DL pipeline for detecting cyberattacks on industrial control systems using an LSTM Autoencoder trained on PLC telemetry data.\n\n## Pipeline Overview\n1. **Data Generation** — Synthetic normal + attack telemetry from PLC simulation\n2. **Preprocessing** — Feature scaling, sliding window sequences\n3. **Model Architecture** — LSTM Encoder-Decoder autoencoder\n4. **Training** — Reconstruct normal operation patterns\n5. **Evaluation** — Detection rates across 8 ICS attack types\n6. **Visualization** — ROC curves, confusion matrices, feature importance\n7. **Real-time Inference** — API integration demo\n\n### Feature Vector (14 dimensions per timestep)\n| # | Feature | Type | Description |\n|---|---------|------|-------------|\n| 0 | conveyor_running | bool | Main conveyor motor state |\n| 1 | production_rate | float | Bottles/min throughput |\n| 2 | reject_rate | float | Reject percentage |\n| 3 | in_flight_bottles | int | Bottles in transit |\n| 4-6 | bottle_at_* | bool | Station presence sensors |\n| 7-8 | output_* | bool | PLC alarm/reject outputs |\n| 9 | network_packet_rate | float | Modbus/TCP packets per second |\n| 10 | network_burst_ratio | float | Traffic burst ratio (0-1) |\n| 11 | scan_time_ms | float | PLC scan cycle time |\n| 12-13 | io_*_sum | int | Total active I/O counts |

## 1. Setup & Imports

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join('..', 'backend')))

import numpy as np
import torch
import matplotlib.pyplot as plt
import matplotlib
from sklearn.metrics import roc_curve, auc, confusion_matrix, classification_report

# Dark theme matching the PLC Emulator UI
matplotlib.rcParams.update({
    'figure.facecolor': '#0f172a', 'axes.facecolor': '#1e293b',
    'text.color': '#e2e8f0', 'axes.labelcolor': '#94a3b8',
    'xtick.color': '#64748b', 'ytick.color': '#64748b',
    'axes.edgecolor': '#334155', 'grid.color': '#334155', 'grid.alpha': 0.5,
})

from app.ml.lstm_autoencoder import (
    generate_normal_data, generate_attack_data,
    train_lstm_autoencoder, create_sequences,
    FeatureScaler, LSTMAutoencoder, FEATURE_NAMES, N_FEATURES,
    load_lstm_artifact, save_lstm_artifact,
)

print(f"PyTorch: {torch.__version__}")
print(f"Features: {N_FEATURES} — {', '.join(FEATURE_NAMES[:7])}...")
print(f"Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")

## 2. Data Generation & Exploration

Generate synthetic telemetry for normal operation and 8 ICS attack scenarios.

In [ ]:
# Generate normal baseline data
normal_data = generate_normal_data(n_samples=8000, seed=42)
print(f"Normal data shape: {normal_data.shape}")

# Generate attack data for each type
ATTACK_TYPES = [
    "dos_flood", "mitm", "false_data_injection", "modbus_injection",
    "stuxnet_like", "sensor_jamming", "replay", "combined",
]
attack_datasets = {}
for at in ATTACK_TYPES:
    attack_datasets[at] = generate_attack_data(n_samples=1000, attack_type=at)
    print(f"  {at}: {attack_datasets[at].shape}")

# Visualize feature distributions: Normal vs DoS vs MITM
fig, axes = plt.subplots(3, 5, figsize=(18, 8))
fig.suptitle("Feature Distributions: Normal vs Attack", fontsize=14, color='#a78bfa')
for i, ax in enumerate(axes.flat):
    if i >= N_FEATURES:
        ax.set_visible(False)
        continue
    ax.hist(normal_data[:, i], bins=30, alpha=0.6, color='#22c55e', label='Normal', density=True)
    ax.hist(attack_datasets['dos_flood'][:, i], bins=30, alpha=0.5, color='#ef4444', label='DoS', density=True)
    ax.hist(attack_datasets['mitm'][:, i], bins=30, alpha=0.4, color='#f59e0b', label='MITM', density=True)
    ax.set_title(FEATURE_NAMES[i], fontsize=7, color='#94a3b8')
    ax.tick_params(labelsize=6)
    if i == 0:
        ax.legend(fontsize=6)
plt.tight_layout()
plt.show()

## 3. Model Training

Train the LSTM Autoencoder on normal-operation data only. The model learns to reconstruct normal patterns — high reconstruction error signals anomaly.

In [ ]:
# Train the LSTM Autoencoder
artifact = train_lstm_autoencoder(
    normal_data,
    seq_len=30, hidden_dim=64, latent_dim=16, n_layers=2,
    epochs=50, batch_size=64, learning_rate=1e-3,
    threshold_quantile=0.98, verbose=True,
)

# Plot training loss curve
plt.figure(figsize=(10, 4))
plt.plot(artifact['epoch_losses'], color='#a78bfa', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.title('Training Loss Curve', color='#a78bfa')
plt.grid(True)
plt.tight_layout()
plt.show()

meta = artifact['metadata']
print(f"\nModel: {meta['model_version']}")
print(f"Threshold: {meta['threshold']:.6f}")
print(f"Final loss: {meta['final_train_loss']:.6f}")
print(f"Train time: {meta['train_time_seconds']}s")

## 4. Evaluation — Detection Rates & ROC Curves

Evaluate the trained model against 8 ICS/SCADA attack types and compute per-attack detection rates, ROC curves, and a multi-class confusion matrix.

In [ ]:
# Load model for evaluation
scaler = FeatureScaler.from_dict(artifact['scaler'])
metadata = artifact['metadata']
seq_len = metadata['seq_len']
threshold = metadata['threshold']

model = LSTMAutoencoder(
    input_dim=N_FEATURES, hidden_dim=metadata['hidden_dim'],
    latent_dim=metadata['latent_dim'], seq_len=seq_len, n_layers=metadata['n_layers'],
)
model.load_state_dict(artifact['state_dict'])
model.eval()

def compute_errors(data):
    scaled = scaler.transform(data)
    seqs = create_sequences(scaled, seq_len)
    tensor = torch.from_numpy(seqs)
    with torch.no_grad():
        recon = model(tensor)
        return torch.mean((recon - tensor) ** 2, dim=(1, 2)).cpu().numpy()

# Normal baseline errors
normal_errors = compute_errors(generate_normal_data(1000, seed=999))

# Per-attack errors + ROC data
fig, axes = plt.subplots(2, 4, figsize=(18, 8))
fig.suptitle("ROC Curves per Attack Type", fontsize=14, color='#a78bfa')
colors = ['#ef4444','#f59e0b','#22c55e','#3b82f6','#a78bfa','#ec4899','#06b6d4','#f97316']

all_y_true, all_y_scores = [], []
detection_rates = {}

for idx, (attack, color) in enumerate(zip(ATTACK_TYPES, colors)):
    attack_errors = compute_errors(attack_datasets[attack])
    
    y_true = np.concatenate([np.zeros(len(normal_errors)), np.ones(len(attack_errors))])
    y_scores = np.concatenate([normal_errors, attack_errors])
    all_y_true.extend(y_true)
    all_y_scores.extend(y_scores)
    
    fpr, tpr, _ = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)
    det_rate = (attack_errors >= threshold).sum() / len(attack_errors) * 100
    detection_rates[attack] = det_rate
    
    ax = axes[idx // 4][idx % 4]
    ax.plot(fpr, tpr, color=color, linewidth=2, label=f'AUC={roc_auc:.3f}')
    ax.plot([0,1], [0,1], '--', color='#475569', linewidth=0.8)
    ax.fill_between(fpr, tpr, alpha=0.15, color=color)
    ax.set_title(f"{attack} ({det_rate:.0f}%)", fontsize=9, color='#94a3b8')
    ax.legend(fontsize=8, loc='lower right')
    ax.set_xlim(0, 1); ax.set_ylim(0, 1.02)
    ax.tick_params(labelsize=7)

plt.tight_layout()
plt.show()

# Summary bar chart
fig, ax = plt.subplots(figsize=(12, 4))
bars = ax.barh(list(detection_rates.keys()), list(detection_rates.values()), color=colors)
ax.set_xlabel('Detection Rate (%)')
ax.set_title('Attack Detection Rates', color='#a78bfa')
ax.set_xlim(0, 105)
for bar, rate in zip(bars, detection_rates.values()):
    ax.text(bar.get_width() + 1, bar.get_y() + bar.get_height()/2, f'{rate:.1f}%', 
            va='center', fontsize=9, color='#e2e8f0')
ax.axvline(x=90, color='#22c55e', linestyle='--', alpha=0.5, label='90% target')
ax.legend(fontsize=8)
plt.tight_layout()
plt.show()

avg_det = np.mean(list(detection_rates.values()))
fpr_normal = (normal_errors >= threshold).sum() / len(normal_errors) * 100
print(f"\nAverage Detection Rate: {avg_det:.1f}%")
print(f"False Positive Rate: {fpr_normal:.1f}%")

## 5. Feature Importance & Error Analysis

Analyze which features contribute most to anomaly detection for each attack type.

In [ ]:
# Per-feature error heatmap across attack types
feature_error_matrix = np.zeros((len(ATTACK_TYPES), N_FEATURES))

for i, attack in enumerate(ATTACK_TYPES):
    data = attack_datasets[attack]
    scaled = scaler.transform(data)
    seqs = create_sequences(scaled, seq_len)
    tensor = torch.from_numpy(seqs)
    with torch.no_grad():
        recon = model(tensor)
        per_feat = torch.mean((recon - tensor) ** 2, dim=(0, 1)).cpu().numpy()
        feature_error_matrix[i] = per_feat

fig, ax = plt.subplots(figsize=(14, 6))
im = ax.imshow(feature_error_matrix, aspect='auto', cmap='magma')
ax.set_xticks(range(N_FEATURES))
ax.set_xticklabels([f.replace('_', '\n') for f in FEATURE_NAMES], fontsize=7, rotation=45, ha='right')
ax.set_yticks(range(len(ATTACK_TYPES)))
ax.set_yticklabels(ATTACK_TYPES, fontsize=9)
ax.set_title('Per-Feature Reconstruction Error by Attack Type', color='#a78bfa', fontsize=13)
plt.colorbar(im, ax=ax, label='MSE')

# Annotate cells
for i in range(len(ATTACK_TYPES)):
    for j in range(N_FEATURES):
        val = feature_error_matrix[i, j]
        color = '#e2e8f0' if val < feature_error_matrix.max() * 0.5 else '#0f172a'
        ax.text(j, i, f'{val:.3f}', ha='center', va='center', fontsize=6, color=color)

plt.tight_layout()
plt.show()

## 6. Confusion Matrix & Classification Report

Binary classification: Normal (0) vs Anomaly (1) using the learned threshold.

In [ ]:
# Combine all attack errors for binary classification
all_y_true = np.array(all_y_true)
all_y_scores = np.array(all_y_scores)
all_y_pred = (all_y_scores >= threshold).astype(int)

# Confusion Matrix
cm = confusion_matrix(all_y_true, all_y_pred)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot confusion matrix
im = ax1.imshow(cm, cmap='Blues')
ax1.set_xticks([0, 1]); ax1.set_yticks([0, 1])
ax1.set_xticklabels(['Normal', 'Anomaly'])
ax1.set_yticklabels(['Normal', 'Anomaly'])
ax1.set_xlabel('Predicted'); ax1.set_ylabel('Actual')
ax1.set_title('Confusion Matrix', color='#a78bfa')
for i in range(2):
    for j in range(2):
        ax1.text(j, i, f'{cm[i,j]}', ha='center', va='center', fontsize=16, 
                color='#0f172a' if cm[i,j] > cm.max()/2 else '#e2e8f0')

# Overall ROC
fpr_all, tpr_all, _ = roc_curve(all_y_true, all_y_scores)
roc_auc_all = auc(fpr_all, tpr_all)
ax2.plot(fpr_all, tpr_all, color='#a78bfa', linewidth=2.5, label=f'Overall AUC = {roc_auc_all:.4f}')
ax2.plot([0,1], [0,1], '--', color='#475569')
ax2.fill_between(fpr_all, tpr_all, alpha=0.15, color='#a78bfa')
ax2.set_xlabel('False Positive Rate'); ax2.set_ylabel('True Positive Rate')
ax2.set_title('Overall ROC Curve', color='#a78bfa')
ax2.legend(fontsize=11)
ax2.set_xlim(0, 1); ax2.set_ylim(0, 1.02)

plt.tight_layout()
plt.show()

# Classification report
print(classification_report(all_y_true, all_y_pred, target_names=['Normal', 'Anomaly']))

## 7. Model Architecture Summary

Inspect the LSTM Autoencoder architecture and parameter count.

In [ ]:
# Model architecture summary
print("LSTM Autoencoder Architecture")
print("=" * 50)
print(model)
print("=" * 50)
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters:     {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Model size:           {total_params * 4 / 1024:.1f} KB (float32)")
print(f"\nHyperparameters:")
for k in ['seq_len','hidden_dim','latent_dim','n_layers','epochs','batch_size','learning_rate']:
    print(f"  {k}: {metadata[k]}")

In [ ]:
## 8. Save & Deploy

Save the trained artifact for use with the FastAPI backend inference endpoint.

In [ ]:
# Save the trained model artifact
output_path = os.path.join('..', 'backend', 'models', 'lstm_anomaly_detector.pt')
save_lstm_artifact(artifact, output_path)

import pathlib
size_kb = pathlib.Path(output_path).stat().st_size / 1024
print(f"Artifact saved to: {output_path}")
print(f"File size: {size_kb:.0f} KB")
print(f"\nTo use with the backend API:")
print(f"  export LSTM_MODEL_PATH=/app/models/lstm_anomaly_detector.pt")
print(f"  # or set in docker-compose.yml environment")
print(f"\nAPI endpoints:")
print(f"  POST /anomaly/score   — ingest + score telemetry")
print(f"  POST /anomaly/ingest  — buffer telemetry only")
print(f"  GET  /anomaly/status  — model status")
print(f"  GET  /anomaly/history — score history")